<h1> Rank Determination

In this two part tutorial, we demonstrate how we can construct polynomial approximants using effectively subsampled quadratures with gradient evaluations of a model. 

In [1]:
%matplotlib inline
from equadratures import *
import numpy as np

In [2]:
cond = np.zeros((8, 100))
for i in range(2, 10):
    value = i
    x1 = Parameter(param_type="Uniform", lower=-1, upper=1, points=value, derivative_flag=1)
    x2 = Parameter(param_type="Uniform", lower=-1, upper=1, points=value, derivative_flag=1)
    parameters = [x1, x2]
    hyperbolic_cross = IndexSet("Hyperbolic basis", orders=[value-1,points-1], q=1.0)
    
    # Create ESQ object
    esq = EffectiveSubsampling(parameters, hyperbolic_cross)
    esq.set_no_of_evals(evals)
    cond[i, j] = np.linalg.matrix_rank(np.mat( np.vstack([esq.A_subsampled, esq.C_subsampled]), dtype='float64') )
    

Now select a hyperbolic basis

The total number of basis terms in this basis is:

120


Now, first lets assume that we are permitted "terms" function evaluations (as many as the number of basis terms). In that case, we wish to determine our coefficient vector. This will be used for an error analysis later on!

In [21]:
esq.set_no_of_evals(terms)
x =  esq.computeCoefficients(fun)

Now, assuming we have gradient evaluations, what is the least number of function evaluations we will require? 

In [18]:
evals = 51
print evals
esq.set_no_of_evals(evals)
print np.linalg.matrix_rank(np.mat( np.vstack([esq.A_subsampled, esq.C_subsampled]), dtype='float64') )
print '********'
x1 =  esq.computeCoefficients(fun, fungrad,'weighted')
x1b =  esq.computeCoefficients(fun, fungrad,'equality')
weighted_error = np.linalg.norm(x-x1, 2)
equality_error = np.linalg.norm(x-x1b, 2)

51
51
97
120
********


In [19]:
print weighted_error, equality_error

2.25639578224e-09 2.25980496072e-09


Now start pruning down the columns

In [20]:
esq.set_no_of_evals(evals)
cols_to_prune = 5
esq.prune(cols_to_prune)
x1 =  esq.computeCoefficients(fun, fungrad,'weighted')
x1b =  esq.computeCoefficients(fun, fungrad,'equality')
xnew =  x[0:(terms-cols_to_prune)]
weighted_error = np.linalg.norm(xnew-x1, 2)
equality_error = np.linalg.norm(xnew-x1b, 2)
print weighted_error, equality_error

3.02515944567e-10 3.02485036368e-10
